# SEP Distribution Tool

This tool downloads SEP intensity-time series data from various different spacecraft and visualizes the SEP distribution using Gaussian curves in one final results plot.

This tool uses a preset proton energy range to keep the various observations comparable.

There is an option to also show a spacecraft-constellation plot (Solar-MACH) and a table summarizing the spacecraft coordinates for the selected time interval.

In [ ]:
# Import modules
import JTL_SEP_functions as jtl

from seppy.util import jupyterhub_data_path
import datetime as dt
import numpy as np
import os
import pandas as pd

from solarmach import SolarMACH

## Saving figures and date

You can usually save a figure from the Notebook by right-clicking on it while holding down the ⇧ Shift key, then select "Save Image As..." (or similar).

In [ ]:
# Set your local path where you want to save the data files. 
# If run on the project's JupyterHub server, set it to a common data folder. 
data_path = f"{os.getcwd()}{os.sep}data/"
data_path = jupyterhub_data_path(data_path)

## Define the event details

Collect the event start and end dates (specifying the start at / near the observed flare onset time), and the eruption location (in Stonyhurst).

In [ ]:
startdate = dt.datetime(2021,5,28,22,19)
enddate = startdate + pd.Timedelta(days=1)
dates = [startdate, enddate]

source_location = [67, 19] #longitude, latitude

In [ ]:
# Options
event_options = {'04Jan2025': {'date': "2025/01/04 18:27:00",
                               'source': [60, -15]},
                 '17Dec2024': {'date': "2024/12/17 12:53:00",
                               'source': [33, -16]},
                 '08Dec2024': {'date': "2024/12/08 08:50:00",
                               'source': [52, -6]},
                 '03Oct2024': {'date': "2024/10/03 12:08:00",
                               'source': [8, -15]},
                 '01Sep2024': {'date': "2024/09/01 14:44:00",
                               'source': [66, -12]}}

## Show the fleet distribution
For more information on the Solar-MACH tool, see: reflinkhere

NB: If you wish to use BepiColombo data then it will need to downloaded separately and saved to the same folder.

In [ ]:
solarmach_table = jtl.solarmach_basic(startdate, data_path, coord_sys='Stonyhurst', source_location=source_location)
display(solarmach_table)        

## Spacecraft options

Here details the relevant spacecraft, their instruments, which channel(s) is(are) used, and the intercalibration values. The only intercalibration value we have found so far is from [Richardson et al. (2014), page 3064](https://doi.org/10.1007/s11207-014-0524-8) which found SOHO/ERNE-HED 13.8-24.2 MeV proton intensities to be about 1.5 times the STEREO-A/HET proton intensities of the same energy.

In [ ]:
## 25-40 MeV Proton channels
#proton_channels = {'PSP': {'instrument': 'EpiHi-HET',
#                           'channels': [8,9],
#                           'intercalibration': 1},
#                   'SOHO': {'instrument': 'ERNE-HED',
#                            'channels': [3,4],
#                            'intercalibration': 1},
#                   'STEREO-A': {'instrument': 'HET',
#                                'channels': [5,8],
#                                'intercalibration': 1},
#                   'Solar Orbiter': {'instrument': 'HET',
#                                     'channels': [19,24],
#                                     'intercalibration': 1}}

# ~14 MeV Proton channels
proton_channels = {'PSP': {'instrument': 'EpiHi-HET',
                           'channels': [3,4],
                           'intercalibration': 1},
                   'SOHO': {'instrument': 'ERNE-HED',
                            'channels': [0],
                            'intercalibration': 0.67},
                   'STEREO-A': {'instrument': 'HET',
                                'channels': [0],
                                'intercalibration': 1},
                   'Solar Orbiter': {'instrument': 'HET',
                                     'channels': [10,12],
                                     'intercalibration': 1}}

Which instruments would you like to include?

In [ ]:
spacecraft = ['PSP', 'SOHO', 'STEREO-A', 'Solar Orbiter']
sc_to_plot = spacecraft # if a spacecraft is removed from the above, but you still want it plotted.
intercalibration = False
radial_scaling = False
radscaling_values = [1.97, 0.27] # values for a \pm b ; 'p': {'a': 1.97, 'b': 0.27}
resampling = '15min'

In [ ]:
df = jtl.load_sc_data(spacecraft=spacecraft, 
                      proton_channels=proton_channels,
                      dates=dates,
                      data_path=data_path,
                      intercalibration=intercalibration, 
                      radial_scaling=radial_scaling,
                      resampling=resampling,
                      reference_loc=source_location)

In [ ]:
df
jtl.plot_timeseries_result(df, data_path, dates)


# Build to one df (synced the times) with headers:
# Layer 1: sc-ins
# Layer 2: Flux, Uncertainty, radial position, longitude.

## Instrument intercalibration

There aren't many recent studies done on calculating the proton instruments intercalibration but most HET instruments are assumed to be similar. The default values used here work on that assumption and use the [Richardson et al.(2014)](https://link.springer.com/article/10.1007/s11207-014-0524-8) study for the intercalibration factor between SOHO-ERNE/HED and STEREO-A HET (1 : 1.5).

def intercalibration_calculation(df, observer_metadict, data_path, dates):
    # Iterate through the observers (first header in df)
    for obs, meta_data in observer_metadict.items():
        factor = meta_data['intercalibration'] # extract the intercalibration factor
        print(obs)
        print(meta_data)
        print(factor)
        jax=input('yes? ')

        # Apply the scaling to the Flux and Uncertainty columns
        for col in ['Flux','Uncertainty']: # Both are calculated the same
            print(df[(obs,col)])
            jax=input('huh?')
            df[(obs, col)] *= factor

    df.to_csv(f"{data_path}SEP_intensities_{dates[0].strftime("%d%m%Y")}_IC.csv") # Save for sanity checks

    return df

In [ ]:
df1 = jtl.intercalibration_calculation(df, proton_channels, data_path, dates)
jtl.plot_timeseries_result(df1, data_path, dates)


In [ ]:
df1

## Radial Scaling

Using the values presented in [Farwa, et al. (2025)](https://www.aanda.org/articles/aa/abs/2025/01/aa50945-24/aa50945-24.html), which used values for 27-37 MeV protons from [Lario et al. (2006)](https://iopscience.iop.org/article/10.1086/508982) (for ~100 keV electrons, [Rodríguez-García et al. (2023)](https://www.aanda.org/10.1051/0004-6361/202244553) is used).

The scaled intensity is calculated as $I_{1 au} = I \cdot R^{a\pm b}$, where $R$ is the radial distance, $I$ is the original intensity, and (for protons specifically) the scaling factors are given as $a \pm b = 1.97 \pm 0.27$.

To calculate the scaled uncertainty, we use the following procedure:
1. Calculate the boundary limits for the intensity calculation (e.g. $I_+ = I\cdot R^{a+b}$; $I_- = I\cdot R^{a-b}$; Therefore, $\Delta I_+ = |I_{1 au}-I_+|$ and $\Delta I_- = |I_{1 au}-I_-|$ are the limits.
2. Find the higher boundary limit, as long as it is < the nominal value ($I_{1 au}$).
3. Calculate the scaled uncertainty value: $\Delta I_{1 au} = \Delta I \cdot R^a$.
4. Combine both to get a final uncertainty value: $\Delta I_{1 au, final} = \sqrt{(\beta)^2 + (\Delta I_{1 au})^2}$.

NB: Check that this final uncertainty is still less than the intensity value!

def radial_scaling_calculation(df0, data_path, scaling_values, dates):
    df = df0.copy(deep=True) # so it doesnt mess with the OG df's
    
    a = scaling_values[0]
    b = scaling_values[1]

    # Iterate through observers
    for obs, df_obs in df.groupby(level=0, axis=1): # returning the observer and their specific df

        for t in df_obs.index:
            print(df_obs.loc[t, (obs,'Flux')])
            if pd.isna(df_obs.loc[t, (obs,'Flux')]) or pd.isna(df_obs.loc[t, (obs,'r_dist')])\
            or (df_obs.loc[t, (obs,'Flux')]==0) or (df_obs.loc[t, (obs,'r_dist')]==0):
                f_rscld = np.nan
                unc_final = np.nan
            
            else:
                # Scale the flux
                f_rscld = df_obs.loc[t, (obs,'Flux')] * (df_obs.loc[t, (obs,'r_dist')] ** a)

                # Scale the uncertainty
                ## Find the difference from the boundaries
                unc_plus = df_obs.loc[t, (obs,'Flux')] * (df_obs.loc[t, (obs,'r_dist')] **(a+b))
                unc_limit_plus = abs(f_rscld - unc_plus)
                unc_minus = df_obs.loc[t, (obs,'Flux')] * (df_obs.loc[t, (obs,'r_dist')] **(a-b))
                unc_limit_minus = abs(f_rscld - unc_minus)
    
                if (unc_limit_plus >= unc_limit_minus) and (f_rscld - unc_limit_plus > 0):
                    chosen_unc_limit = unc_limit_plus
                elif (unc_limit_minus >= unc_limit_plus) and (f_rscld - unc_limit_minus > 0):
                    chosen_unc_limit = unc_limit_minus
                else:
                    print("There's a problem with the limits")
                    print("OG flux: ", df_obs.loc[t, (obs,'Flux')])
                    print("OG rad: ", df_obs.loc[t, (obs,'r_dist')])
                    print("Scaled Flux: ", f_rscld)
                    print("Unc plus: ", unc_limit_plus)
                    print("Unc minus: ", unc_limit_minus)
                    jax = input('Continue? ')
                    chosen_unc_limit = np.nan
    
                ## Find the calculated scaled uncertainty
                unc_calculated = df_obs.loc[t, (obs,'Uncertainty')] * (df_obs.loc[t, (obs,'r_dist')] ** a)
    
                ## Merge both results for the final scaled uncertainty
                unc_final = np.sqrt((unc_calculated)**2 + (chosen_unc_limit)**2)

            df.loc[t, (obs, 'Flux')] = f_rscld
            df.loc[t, (obs, 'Uncertainty')] = unc_final


    df.to_csv(f"{data_path}SEP_intensities_{dates[0].strftime("%d%m%Y")}_RS.csv") # Save for sanity checks
    return df

In [ ]:
df2 = jtl.radial_scaling_calculation(df1, data_path, radscaling_values, dates)
jtl.plot_timeseries_result(df2, data_path, dates)


## Background subtraction

plot time series of all instruments and let the user decide on a background window for each. Then background subtract it all.

In [ ]:
def background_subtracting(df, data_path, background_window):
    """Input: the full dataframe, the path for saving files, the background window.
    Process: 1. Find the nanmean  and nanstd of the background window for both flux and unc.
            2. Subtract the full column by the [avg - std] (this way there's less chance to get half the values as nan).
                - The unc is calculated as: unc_adj = sqrt(unc**2 + [avg-std]**2)
            3. Return the updated df."""
    # Iterate through the big df
    for obs, obs_df in df.groupby(level=0, axis=1): # Returns the observer and their specific df
        print(obs)
        print(obs_df.head())
        # A list of just the values within the background window
        bg_flux = obs_df[(obs,'Flux')][background_window[0]:background_window[1]]
        bg_func = obs_df[(obs,'Uncertainty')][background_window[0]:background_window[1]]

        # Find the nanmean and nanstd
        f_bg_avg = float(np.nanmean(bg_flux)) - float(np.nanstd(bg_flux, ddof=1))
        u_bg_avg = float(np.nanmean(bg_func)) - float(np.nanstd(bg_func, ddof=1))
        print('flux bg avg - std: ', f_bg_avg)
        print('unc bg avg - std: ', u_bg_avg)
        jax=input('good?')

        # Adjust the whole column
        adj_flux = (obs_df.loc[:, (obs,'Flux')]) - f_bg_avg
        adj_func = np.sqrt( ((obs_df.loc[:, (obs,'Uncertainty')])**2) + (u_bg_avg**2) )

        # Replace any negative results with 'nan'
        adj_flux = adj_flux.where(adj_flux>=0, np.nan) # new list = old list where the values are >=0, else make the value 'nan'
        adj_func = adj_func.where(adj_func>=0, np.nan)

        # Put the adjusted column in
        df[(obs,'Flux')] = adj_flux
        df[(obs,'Uncertainty')] = adj_func

    df.to_csv(data_path+'backsubtest.csv', na_rep='nan')
    
    return df
        
        















    

In [ ]:
# Plot

# User input on background
background_window = [startdate-dt.timedelta(hours=2), startdate+dt.timedelta(minutes=60)]
jtl.plot_timeseries_result(df2, data_path, dates, background_window=background_window)

In [ ]:
# Background subtraction
print(background_window)
df3 = background_subtracting(df2, data_path, background_window)

In [ ]:
jtl.plot_timeseries_result(df3, data_path, dates, background_window=background_window)

# Gaussian curve fitting
First with scipy.curve_fit then with scipy's ODR function (with the uncertainties). Produce and save a fig into a new subfolder each time. Fig should include curve on the left and intensity with vertical line for time tracking on the right.

# Plot final time series
3 subplots:
1. Intensity
2. Center
3. Sigma

# (Optional) Gif of gaussian figs

# (For later) Latitude fits 
Fit ecliptic plane instruments first (instruments at < 10 degrees latitude) then plot all instruments with ecliptic gaussian to deduce offset. Still to find events for this.